In [1]:
##########################################################
# Q-learning implementation with 3D Item Sets, game has 3 states 0, 1, 2
# 1. split train data into training set and test set
# 2. train Q-Tables on Training set
# 3. make suggestions for test set
# 4. Calculate Metrics 1 for our suggestions
# 5. Make prediction for the competition's test set
#########################################################

In [13]:
# 1. Split Train 
from DataPrep import *
userFeaturesTrain, recItemsTrain, purchaseLabelTrain, userFeaturesVal, recItemsVal, purchaseLabelVal = splitTrainSet()
userFeaturesTrain = pd.concat((userFeaturesTrain, userFeaturesVal), ignore_index=True)
recItemsTrain = np.vstack((recItemsTrain, recItemsVal))
purchaseLabelTrain = np.vstack((purchaseLabelTrain, purchaseLabelVal))

In [14]:
# dimension reduction with PCA
# comment this part out to use original user features 

# cluster model of 20D
from DataPrep import getClusterModel200_20D, getClusterLabels100
ClusterModel, clusterLabels = getClusterLabels100()

from DataPrep import getPCATransformer
PCAtransformer = getPCATransformer()
# userFeaturesTrain = PCAtransformer.transform(userFeaturesTrain)
# userFeaturesVal = PCAtransformer.transform(userFeaturesVal)


/home/minh/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator SparsePCA from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [15]:
# 2.
import numpy as np
from tqdm import tqdm
from classes.QLearning2 import *

# predict user cluster label for users in the training set
NCLUSTERS = 100
# ClusterModel, clusterLabels = getClusterLabels100()
clusterLabelTrain  = ClusterModel.predict(userFeaturesTrain)
# load item info
from classes.Items import Items
itemInfo = Items()
# load itemset info
from classes.ItemSet import ItemSet3
itemSets = ItemSet3()
N_Sets = itemSets.getNSets()
print('Number of Item Sets 3: ' + str(N_Sets))

# initialize Q tables 
QLModels = []
trainData = []
for i in range(NCLUSTERS):
    QLModels.append(QLearning(n_states = 3, n_actions = N_Sets))
    trainData.append([])

# to train Q tables: 
#### state: 0, 1, or 2
#### action: the itemSet recommended
#### reward: (item is purchased) * price
#### nextState: 1 or 2. -1 if there is no next state
#### to feed a set of (state, action, reward) to a Q table
for i in tqdm(range(len(recItemsTrain))):
# loop thru samples
    recItems = recItemsTrain[i]
    purLabel = purchaseLabelTrain[i]
    for j in [0, 3, 6]: # process each Set3 at once
        if j>2 and purLabel[0]*purLabel[1]*purLabel[2]==0:
            # don't train if game stopped
            break
        if j>5 and purLabel[3]*purLabel[4]*purLabel[5]==0:
            # don't train if game stopped
            break
        # calculate state:
        state = j/3
        # next state:
        if j==0:
            if purLabel[0]*purLabel[1]*purLabel[2]==0: # terminated
                nextState = -1
            else:
                nextState = 1
        elif j==3:
            if purLabel[3]*purLabel[4]*purLabel[5]==0: # terminated
                nextState = -1
            else:
                nextState = 2
        else:
            nextState = -1
        
        # calculate action:
        itemSet = [recItems[j], recItems[j+1], recItems[j+2]]
        action = itemSets.getSetID(itemSet)

        # calculate rewards, note: itemPrice is an array, itemID from raw data is 1-based index
        prices = [itemInfo.getItemPrice(itemSet[0]), itemInfo.getItemPrice(itemSet[1]), itemInfo.getItemPrice(itemSet[2])]
        labels = [purLabel[j], purLabel[j+1], purLabel[j+2]]
        reward = sum([prices[t]*labels[t] for t in range(3)])

        train_data = (state, action, reward, nextState)
        # predict user cluster label of this sample based on user features
        clusterID = clusterLabelTrain[i]
        trainData[clusterID].append(train_data)


  0%|          | 0/260087 [00:00<?, ?it/s]

Number of Item Sets 3: 112368


100%|██████████| 260087/260087 [00:15<00:00, 16636.62it/s]


In [16]:
for i in tqdm(range(NCLUSTERS)):
    QLModels[i].trainParallel(trainData[i])

100%|██████████| 100/100 [04:37<00:00,  2.77s/it]


In [17]:
# 3. make suggestion for Val set
# predict user cluster label for users in the Val set
clusterLabelVal  = ClusterModel.predict(userFeaturesVal)

for i in range(NCLUSTERS):
    QLModels[i].initPredCache()


# make suggestion for each test sample
items_out_val = []
for cluster in tqdm(clusterLabelVal):
# clusterLabelTest is array of labels of users in Val set
    model = QLModels[cluster]
    bestSetID0 = model.predictBestK(0, 2)   # best action for first state
    bestSetID1 = model.predictBestK(1, 100)   # best 100 actions for second state
    bestSetID2 = model.predictBestK(2, 100)   # best 100 actions for third state
    finalItems = []
    items = itemSets.getItemSet(bestSetID0[0])
    finalItems.extend(list(items))
    for setID in bestSetID1:  # make suggestion for stage 1
        items = itemSets.getItemSet(setID)
        if items[0] not in finalItems and items[1] not in finalItems and items[2] not in finalItems:
            finalItems.extend(list(items))
            break
    for setID in bestSetID2:  # make suggestion for stage 2
        items = itemSets.getItemSet(setID)
        if items[0] not in finalItems and items[1] not in finalItems and items[2] not in finalItems:
            finalItems.extend(list(items))
            break
    items_out_val.append(finalItems)


100%|██████████| 52018/52018 [00:09<00:00, 5745.16it/s] 


In [6]:
# Calculate Metrics 1 for our suggestions

In [20]:
# 4. calculate metrics
from classes.Metrics import *
metrics = Metrics(recItemsVal, purchaseLabelVal)
score = metrics.calculate_metrics2(items_out_val, w3 = 1000)
print(score)
# calculate metrics of test set (max score possible by ground truth)
score_max = metrics.calculate_metrics2(recItemsVal, w3 = 1000)
print(score_max) # max score possible
print('percentage of max score: ' + str(score/score_max))

38679534946
159313694728
percentage of max score: 0.2427885123877045


In [9]:
# 5. make suggestion for competition's test set
# predict user cluster label for users in the test set
userIDs, userFeaturesTest = getUserFeaturesTestSet()
userFeaturesTest = PCAtransformer.transform(userFeaturesTest)

In [10]:
clusterLabelTest  = ClusterModel.predict(userFeaturesTest)

# make suggestion for each test sample
items_out_test = []
for cluster in tqdm(clusterLabelTest):
# clusterLabelTest is array of labels of users in test set
    model = QLModels[cluster]
    bestSetID0 = model.predictBestK(0, 2)   # best action for first state
    bestSetID1 = model.predictBestK(1, 100)   # best 100 actions for second state
    bestSetID2 = model.predictBestK(2, 100)   # best 100 actions for third state
    finalItems = []
    items = itemSets.getItemSet(bestSetID0[0])
    finalItems.extend(list(items))
    for setID in bestSetID1:  # make suggestion for stage 1
        items = itemSets.getItemSet(setID)
        if items[0] not in finalItems and items[1] not in finalItems and items[2] not in finalItems:
            finalItems.extend(list(items))
            break
    for setID in bestSetID2:  # make suggestion for stage 2
        items = itemSets.getItemSet(setID)
        if items[0] not in finalItems and items[1] not in finalItems and items[2] not in finalItems:
            finalItems.extend(list(items))
            break
    items_out_test.append(finalItems)


100% 206096/206096 [10:13:43<00:00,  5.60it/s] 


In [12]:
# write recommended items to output csv file
from classes.output import writeOutput
writeOutput(items_out_test, 'QLearning-3D-20DFeatures.csv', userIDs)

In [11]:
print(items_out_test[:50])


[[1, 16, 26, 51, 80, 107, 160, 200, 234], [1, 7, 14, 51, 80, 107, 172, 199, 234], [1, 16, 26, 51, 80, 107, 160, 200, 234], [1, 17, 33, 51, 80, 107, 172, 233, 234], [1, 7, 14, 73, 78, 125, 158, 171, 214], [1, 17, 30, 51, 80, 107, 172, 233, 234], [1, 13, 15, 73, 79, 126, 171, 200, 220], [1, 7, 24, 47, 79, 107, 160, 200, 221], [1, 17, 30, 79, 80, 132, 172, 199, 221], [6, 7, 14, 79, 101, 129, 172, 196, 234], [1, 7, 28, 45, 49, 112, 164, 199, 200], [1, 7, 14, 51, 80, 107, 172, 199, 220], [1, 7, 32, 73, 78, 107, 172, 188, 234], [1, 5, 34, 50, 79, 101, 164, 234, 235], [1, 7, 14, 51, 80, 107, 172, 199, 234], [1, 7, 32, 61, 80, 106, 172, 199, 235], [1, 4, 14, 80, 106, 111, 160, 200, 233], [1, 7, 14, 41, 80, 101, 164, 172, 213], [4, 15, 20, 73, 86, 132, 164, 200, 234], [1, 19, 21, 77, 80, 106, 164, 200, 234], [1, 7, 28, 73, 79, 100, 172, 196, 233], [1, 7, 14, 51, 80, 107, 172, 199, 220], [1, 7, 18, 61, 80, 107, 171, 200, 220], [1, 7, 14, 40, 47, 79, 160, 199, 213], [1, 17, 30, 61, 80, 107, 172, 

In [4]:
QLModels[0].QTable.dtype

dtype('float32')